# old

In [ ]:
import os
import numpy as np
from PIL import Image
import astra
import matplotlib.pyplot as plt

tiff_dir = './projections/'
tiff_files = sorted([f for f in os.listdir(tiff_dir) if f.endswith('.tif')])
projections = np.stack([np.array(Image.open(os.path.join(tiff_dir, f)), dtype=np.float32) for f in tiff_files])

num_projections, num_rows, num_cols = projections.shape
angles = np.linspace(0, np.pi, num_projections, endpoint=False)


In [ ]:
reconstructed_slices = []

for i in range(num_rows):
    # Get sinogram for slice i
    sinogram = projections[:, i, :]

    # Define geometry for this slice
    proj_geom = astra.create_proj_geom('parallel', 1.0, num_cols, angles)
    vol_geom = astra.create_vol_geom(num_cols, num_cols)

    # Create data objects
    sinogram_id = astra.data2d.create('-sino', proj_geom, sinogram)
    rec_id = astra.data2d.create('-vol', vol_geom)

    # Configure FBP algorithm
    cfg = astra.astra_dict('FBP_CUDA')
    cfg['ProjectionDataId'] = sinogram_id
    cfg['ReconstructionDataId'] = rec_id

    alg_id = astra.algorithm.create(cfg)
    astra.algorithm.run(alg_id)

    # Get reconstructed slice
    rec = astra.data2d.get(rec_id)
    reconstructed_slices.append(rec)

    # Clean up
    astra.algorithm.delete(alg_id)
    astra.data2d.delete([sinogram_id, rec_id])

# Convert to 3D volume
reconstructed_volume = np.stack(reconstructed_slices, axis=0)


# new

In [ ]:
import os
import numpy as np
import pydicom
import astra
from tqdm import tqdm

# Directory containing DICOM projection images
dcm_dir = 'C:/Users/fedhila/Desktop/2D Scans/scan_03/DICOM'
dcm_files = sorted([os.path.join(dcm_dir, f) for f in os.listdir(dcm_dir) if f.endswith('.dcm')])

# Load all DICOM projection images into a 3D NumPy array
print("Loading DICOM projection images...")
projections = np.stack([pydicom.dcmread(f).pixel_array.astype(np.float32) for f in tqdm(dcm_files)], axis=0)

# Get number of projections, rows, and cols
num_projections, num_rows, num_cols = projections.shape
print(f"Loaded {num_projections} projections, each with shape {num_rows} x {num_cols}")

# Define angles — assuming evenly spaced projections over 180° (π radians)
angles = np.linspace(0, np.pi, num_projections, endpoint=False)

# Optional: Check if CUDA is available (will be False for you)


# Reconstruct each slice
reconstructed_slices = []

print("Reconstructing slices with FBP (CPU)...")
for i in tqdm(range(num_rows)):
    # Extract the sinogram for slice i
    sinogram = projections[:, i, :]

    # Define geometry for this slice
    proj_geom = astra.create_proj_geom('parallel', 1.0, num_cols, angles)
    vol_geom = astra.create_vol_geom(num_cols, num_cols)

    # Create 2D data objects for CPU
    sinogram_id = astra.data2d.create('-sino', proj_geom, sinogram)
    rec_id = astra.data2d.create('-vol', vol_geom)

    # Configure the FBP algorithm for CPU
    cfg = astra.astra_dict('SIRT')
    cfg['ProjectionDataId'] = sinogram_id
    cfg['ReconstructionDataId'] = rec_id

    # Run the algorithm
    alg_id = astra.algorithm.create(cfg)
    astra.algorithm.run(alg_id)

    # Retrieve the reconstructed slice
    rec = astra.data2d.get(rec_id)
    reconstructed_slices.append(rec)

    # Clean up memory
    astra.algorithm.delete(alg_id)
    astra.data2d.delete([sinogram_id, rec_id])

# Convert list of 2D slices to 3D volume
reconstructed_volume = np.stack(reconstructed_slices, axis=0)

print(f"Reconstruction done. Volume shape: {reconstructed_volume.shape}")

# Visualization

In [7]:
pip install "pyvista[jupyter]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 198.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 305.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 256.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 187.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 253.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 153.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.7/377.7 kB 248.0 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.58.0
    Uninstalling websocket-client-0.58.0:
      Successfully uninstalled websocket-client-0.58.0
  Attempting uninstall: send2trash
    Found existing installation: Send2Trash 1.8.0
    Uninstalling Send2Trash-1.8.0:
      Successfully uninstalled Send2Trash-1.8.0
  Attempting 

In [6]:
import os
import numpy as np
import imageio.v2 as imageio  # Use explicit v2 import
import pyvista as pv
from tqdm import tqdm

# Set PyVista to use trame backend for Jupyter
pv.set_jupyter_backend('trame')

# Path to TIF slices folder
path = r"C:\Users\fedhila\Downloads\tif_slices-20250505T114141Z-001\tif_slices"

# List all tif files, sorted to maintain slice order
tif_files = sorted([os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith(('.tif', '.tiff'))])

if not tif_files:
    print(f"No TIF files found in the directory: {path}")
    exit()

# Load the first image to get dimensions
sample = imageio.imread(tif_files[0])
print(f"Sample image shape: {sample.shape}")

# Load images into a 3D NumPy array
print(f"Loading {len(tif_files)} slices...")
volume = np.stack([imageio.imread(f) for f in tqdm(tif_files)], axis=0)

print(f"Loaded volume shape: {volume.shape}")

# Normalize data to 0-255 range for better visualization
if volume.dtype != np.uint8:
    volume_min = volume.min()
    volume_max = volume.max()
    print(f"Data range: {volume_min} to {volume_max}")
    # Normalize to 0-255 for better visualization
    volume_normalized = ((volume - volume_min) / (volume_max - volume_min) * 255).astype(np.uint8)
else:
    volume_normalized = volume

# Convert to PyVista UniformGrid
# Note: z, y, x ordering for dimensions
nz, ny, nx = volume_normalized.shape
grid = pv.ImageData()  # Use UniformGrid explicitly

# Set the dimensions: shape + 1 because dimensions are points, not cells
grid.dimensions = (nx + 1, ny + 1, nz + 1)

# Set the spacing (voxel size) — adjust if your voxels have physical spacing
grid.spacing = (1, 1, 1)

# Set the origin
grid.origin = (0, 0, 0)

# Add the volume data - use the correct reshape
grid.cell_data["values"] = volume_normalized.flatten(order="F")  # Fortran order for PyVista

print("Creating visualization...")

# Visualize the volume with more options
pl = pv.Plotter()
pl.add_volume(grid, cmap="bone", opacity="sigmoid_5", shade=True)
pl.add_axes()
pl.show()

# Alternative visualization with orthogonal slices
plotter = pv.Plotter()
# Add the volume with a custom transfer function
plotter.add_volume(grid, cmap="bone", opacity="sigmoid_5")
# Add orthogonal slices
slices = grid.slice_orthogonal(x=nx//2, y=ny//2, z=nz//2)
plotter.add_mesh(slices, cmap="bone")
plotter.add_axes()
plotter.show()

ImportError: Please install trame dependencies: pip install "pyvista[jupyter]"